**format_utils.py**

In [ ]:
import pandas as pd
from datetime import datetime

def format_timestamp_for_mysql(iso_timestamp):
    # Reformate en string compatible MySQL
    dt = datetime.strptime(iso_timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")
    return dt.strftime("%Y-%m-%d %H:%M:%S")

def json_to_df(data):
    # Extraction du nom de l'automate, et de ses différents tags
    modbus_data = data["tags"]["modbus_tcp_master"]
    device_name = list(modbus_data.keys())[0]
    entries = modbus_data[device_name]

    # Création du DF
    rows = []

    for tag_name, tag_content in entries.items():
        for value_entry in tag_content["values"]:
            rows.append({
                "table_name": tag_name,
                "timestamp": format_timestamp_for_mysql(value_entry["updateTimeStamp"]),
                "value": value_entry["value"]
            })

    df = pd.DataFrame(rows)
    return df

**sql_utils.py**

In [ ]:
import mysql.connector
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

def connect_db(db_name):
    return mysql.connector.connect(
        host="localhost",      
        user="root",          
        password=os.getenv("PASSWORD", ""), 
        database=db_name,
    )


def insert_data_to_db(df, cursor, connection):
    for _, row in df.iterrows():
        timestamp = row["timestamp"]
        value = row["value"]
        table_name = row["table_name"]
        cursor.execute(
            f"INSERT INTO {table_name} (timestamp, value) VALUES (%s, %s)",
            (timestamp, value)
        )
    connection.commit()


**main.py**

In [ ]:
import paho.mqtt.client as mqtt
import pandas as pd
import json

# Liste des topics mqtt auxquels on va s'abonner (il y en aura d'autres)
topics = [("16_ISHIDA", 0), ("14_CRYOD", 0)]
dataframes = {}

# Fonction appelée lorsqu'on se connecte au broker
def on_connect(client, rc):
    print("Connecté avec le code :", rc)
    client.subscribe(topics)

# Fonction appelée à la réception d’un message
def on_message(msg):
    topic = msg.topic
    payload = msg.payload.decode()
    print(f"Message reçu - Topic: {topic}, Payload: {payload}")
    
    payload = json.loads(payload)
    df = json_to_df(payload)
    
    # Connexion et insertion dans la db
    db = connect_db(topic)
    cursor = db.cursor()
    insert_data_to_db(df, cursor, db)

    cursor.close()
    db.close()

# Déf d'un client mqtt
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

# Connexion au broker local
client.connect("localhost", 1883, 60)

# Boucle réseau pour écouter les messages
client.loop_forever()
